In [1]:
import peakutils
import wfdb
import scipy.signal as signal


from scipy.signal import butter, sosfilt, sosfilt_zi, sosfiltfilt, lfilter, lfilter_zi, filtfilt, sosfreqz, resample
from utils import hamilton_detector, christov_detector, findpeaks, engzee_detector
from ecg_detectors.ecgdetectors import Detectors, MWA, panPeakDetect, searchBack
import os
import numpy as np

# DETECTING THE PEAK

In [2]:
def peak_detection_awdd(series):
    r_peaks=list()
    signal_slice=series
    
    y=smoothner(series)
    
    r_peaks.append(peakutils.indexes(y,thres=0.65,min_dist=50))
    
    return r_peaks
    

    
    
    
    
#For smoothening the noice    

def smoothner(series):
    data=series
    fs = 200  # corresponds to 60 beats per min (normal for human), assumed.
    lowcut = 0.05 * 3.3  # 9.9 beats per min
    highcut = 15
    times = np.arange(data.shape[0], dtype='float') / fs
    
    y = butter_bandpass_forward_backward_filter(series, lowcut, highcut, fs, order=4)
    return y
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    sos = butter(order, [low, high], analog=False, btype="band", output="sos")
    return sos


def butter_bandpass_forward_backward_filter(data, lowcut, highcut, fs, order=5):
    sos = butter_bandpass(lowcut, highcut, fs, order=order)
    y = sosfiltfilt(sos,
                    data)  # Apply a digital filter forward and backward to a signal.This function applies a linear digital filter twice, once forward and once backwards. The combined filter has zero phase and a filter order twice that of the original.
    return y

# DETECTING THE ANOMALOUS POINT

In [3]:
from distance import early_abandoned_euclidean

def find_awdd_detect(series,num_discords):
    discords=list();

    while len(discords) < num_discords:

        bestDiscord = awdd_detect(series)

        if -1 == bestDiscord[0]:
            break

        discords.append(bestDiscord)


    return discords


def awdd_detect(series):
    best_so_far_distance=-1.0
    best_so_far_index=-1.0
    dist=-1.0
    peak_loc={}
    peak_loc=peak_detection_awdd(series)
    num_peak=len(peak_loc)-1
    outer_cnt=0
    

    p=peak_loc[0][0]
    while p<peak_loc[0][num_peak-2]+1:
        nearest_neighbor_distance=np.inf
        outer_len=peak_loc[0][outer_cnt+1]-peak_loc[0][outer_cnt]

        inner_cnt=0
        q=peak_loc[0][0]
        datap=series[peak_loc[0][outer_cnt]:peak_loc[0][outer_cnt+1]]
        while q<peak_loc[0][num_peak-2]+1:
            inner_len=peak_loc[0][inner_cnt+1]-peak_loc[0][inner_cnt]
            dataq=series[peak_loc[0][inner_cnt]:peak_loc[0][inner_cnt+1]]
            if (outer_len>inner_len):
                datap=series[peak_loc[0][outer_cnt]:peak_loc[0][(outer_cnt+1)]-(outer_len-inner_len)]
            if (inner_len>outer_len):
                dataq=series[peak_loc[0][inner_cnt]:peak_loc[0][inner_cnt+1]-(inner_len-outer_len)]
                
            dist = early_abandoned_euclidean(datap, dataq, nearest_neighbor_distance)

            if abs(p-q)>=min(outer_len,inner_len):
                if (~np.isnan(dist)) and (dist < nearest_neighbor_distance):
                    nearest_neighbor_distance = dist

            inner_cnt+=1
            q=peak_loc[0][inner_cnt]

        if ~(np.inf == nearest_neighbor_distance) and (nearest_neighbor_distance > best_so_far_distance):
            best_so_far_distance = nearest_neighbor_distance
            best_so_far_index = p
            
        outer_cnt+=1
        p=peak_loc[0][outer_cnt]
    return best_so_far_index,best_so_far_distance

# TO GET THE SIGNAL DATA

In [4]:
def signal_data():
    records={}
    signal_slice={}
    r_peaks={}
    # flist = os.listdir('/Users/chaitanya/Downloads/ecg-id-database-1.0.0/Person_01/')
    flist = os.listdir('/Users/chaitanya/Downloads/ShubamData/qt-database-1.0.0/')
    flist = [file for file in flist if file.endswith(".dat")]
    for file in flist:
        records[file] = wfdb.rdrecord('/Users/chaitanya/Downloads/ShubamData/qt-database-1.0.0/'+file[:-4],channels=[0])
    for rec in records:
        signal_slice[rec]=np.ndarray.flatten(records[rec].p_signal[0:10000])
    return records,signal_slice

# OBTAINING THE ANOMALOUS POINT FOR EACH RECORD

In [5]:
discords={}

for rec in signal_data()[0]: 
    discords[rec]=find_awdd_detect(signal_data()[1][rec][0:10000],1)

In [6]:
discords

{'sel123.dat': [(1520, 2.7535931071964863)],
 'sel50.dat': [(2513, 1.9928183559973547)],
 'sel44.dat': [(3575, 1.9187691367123874)],
 'sel14046.dat': [(3409, inf)],
 'sele0406.dat': [(5742, 1.1478022477761574)],
 'sele0604.dat': [(8278, 0.3969256857397862)],
 'sel45.dat': [(6612, 2.5229298444467307)],
 'sel51.dat': [(1129, 0.485789735036329)],
 'sel308.dat': [(7714, 1.1051244273836318)],
 'sel47.dat': [(8867, 2.614784885989668)],
 'sele0606.dat': [(1336, 1.109098733206382)],
 'sele0612.dat': [(91, 2.029667460447647)],
 'sele0411.dat': [(5999, 2.1842046607403804)],
 'sele0405.dat': [(3859, 2.0850119903731974)],
 'sele0607.dat': [(1782, 3.8701776445016067)],
 'sel52.dat': [(6700, 1.6631070320337173)],
 'sel46.dat': [(9200, 1.7875332164745918)],
 'sele0203.dat': [(4647, 1.9703933617427762)],
 'sel42.dat': [(6585, 5.2642948245705234)],
 'sele0603.dat': [(2431, 0.43156691254080193)],
 'sel16483.dat': [(6825, 1.0733126291998991)],
 'sele0170.dat': [(9771, 3.9182074983339006)],
 'sel43.dat': 